In [ ]:
%matplotlib inline

import os
import sys
import glob
import time
import numpy as np
import scipy
import scipy.interpolate
import matplotlib
import matplotlib.pyplot as plt
import importlib

# Local modules
import toshiba
import cinterp

# iRadar
sys.path.append(os.path.expanduser('~/Developer/iradar/python'))
import iradar

In [ ]:
files = glob.glob(os.path.expanduser('~/Downloads/toshiba/*.nc'))
filename = files[1]

# azimuth_count = 360
range_count = 200

In [ ]:
plt.rcParams['axes.facecolor'] = 'black'
zmap = iradar.colormap.zmap()
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', zmap[:, :3], N=len(zmap))

In [ ]:
sweep = toshiba.read(filename, maxgate=range_count)

In [ ]:
# Get the reflectivity and raw range and azimuth angles
z = sweep['z']
r = 1.0e-3 * sweep['rr']
a = sweep['aa']

# Only retain the 360-deg coverage, take out the extra piece
k = next(k for k, ia in enumerate(a[::-1]) if abs(ia > a[0]) < 0.5) 
a = a[:-k] / 180.0 * np.pi
z = sweep['z'][:-k, :]

rr, aa = np.meshgrid(r, a)
xx = rr * np.sin(aa)
yy = rr * np.cos(aa)

In [ ]:
turbines = cinterp.data.toshiba_cells
cells = cinterp.pos2cellid(turbines, a / np.pi * 180.0, r)

a_turb = turbines[:, 0] / 180.0 * np.pi
r_turb = turbines[:, 1]
x_turb = r_turb * np.sin(a_turb)
y_turb = r_turb * np.cos(a_turb)

z_interp = z.copy()

m = np.zeros(z_interp.shape, dtype=bool)
for c in cells:
    m[c[0], c[1]] = True
m1 = cinterp.dilate(m)
m2 = cinterp.dilate(m1)

z_interp[m2] = 36
z_interp[m] = 66.0

# Convert the cells into tagged cells
tags = cinterp.mask2tags(m2)

# Take out the dilated portion
tags[m2 ^ m1] = 0

In [ ]:
xlim = (-1, 6)
ylim = (-12.5, -5.5)

plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, z_interp, cmap=cmap)
plt.plot(x_turb, y_turb, 'xw', markersize=3)
plt.clim((-32, 96))
plt.colorbar()
plt.xlim(xlim)
plt.ylim(ylim)

plt.figure(figsize=(5.25, 4), dpi=200)
plt.pcolormesh(xx, yy, 5 * tags, cmap=cmap)
plt.clim((-32, 96))
plt.colorbar()
plt.xlim(xlim)
plt.ylim(ylim)
plt.show()